In [ ]:
'''
Enable Colab GPU - 
In Runtime dropdown open Change Runtime Type,
Change Hardware accelerator dropdown to GPU
'''

!pip install pyyaml==5.1
!pip install torch==1.8.0+cu111 torchvision==0.9.0+cu111 torchaudio==0.8.0 -f https://download.pytorch.org/whl/torch_stable.html
import torch, torchvision
print(torch.__version__, torch.cuda.is_available())
!gcc --version

In [ ]:
# install detectron2: (Colab has CUDA 10.1 + torch 1.8)
!pip install detectron2 -f https://dl.fbaipublicfiles.com/detectron2/wheels/cu101/torch1.8/index.html
# exit(0)  # After installation, you need to "restart runtime" in Colab. This line can also restart runtime


In [ ]:
# Some basic setup:
# Setup detectron2 logger
import detectron2
import cv2
from detectron2.utils.logger import setup_logger
setup_logger()

# import some common libraries
import numpy as np
import os, json, cv2, random
from google.colab.patches import cv2_imshow

# import some common detectron2 utilities
from detectron2 import model_zoo
from detectron2.engine import DefaultPredictor
from detectron2.config import get_cfg
from detectron2.utils.visualizer import Visualizer
from detectron2.data import MetadataCatalog, DatasetCatalog

import re
import matplotlib.pyplot as plt
import math
from math import atan2, degrees


def angle_between(p1, p2, p3):
    x1, y1 = p1
    x2, y2 = p2
    x3, y3 = p3
    deg1 = (360 + degrees(atan2(x1 - x2, y1 - y2))) % 360
    deg2 = (360 + degrees(atan2(x3 - x2, y3 - y2))) % 360
    return deg2 - deg1 if deg1 <= deg2 else 360 - (deg1 - deg2)


In [ ]:
from IPython.display import display, Javascript
from google.colab.output import eval_js
from base64 import b64decode
import csv

def record_video(filename='video.mp4'):
  js = Javascript("""
    async function recordVideo() {
      // mashes together the advanced_outputs.ipynb function provided by Colab, 
      // a bunch of stuff from Stack overflow, and some sample code from:
      // https://developer.mozilla.org/en-US/docs/Web/API/MediaStream_Recording_API
      const options = { mimeType: "video/webm; codecs=vp9" };
      const div = document.createElement('div');
      const capture = document.createElement('button');
      const stopCapture = document.createElement("button");
      capture.textContent = "Start Recording";
      capture.style.background = "green";
      capture.style.color = "white";

      stopCapture.textContent = "Stop Recording";
      stopCapture.style.background = "red";
      stopCapture.style.color = "white";
      div.appendChild(capture);

      const video = document.createElement('video');
      const recordingVid = document.createElement("video");
      video.style.display = 'block';

      const stream = await navigator.mediaDevices.getUserMedia({video: true});
      // create a media recorder instance, which is an object
      // that will let you record what you stream.
      let recorder = new MediaRecorder(stream, options);
      document.body.appendChild(div);
      div.appendChild(video);
      // Video is a media element.  This line here sets the object which serves
      // as the source of the media associated with the HTMLMediaElement
      // Here, we'll set it equal to the stream.
      video.srcObject = stream;
      // We're inside an async function, so this await will fire off the playing
      // of a video. It returns a Promise which is resolved when playback has 
      // been successfully started. Since this is async, the function will be 
      // paused until this has started playing. 
      await video.play();

      // Resize the output to fit the video element.
      google.colab.output.setIframeHeight(document.documentElement.scrollHeight, true);
      // and now, just wait for the capture button to get clicked in order to
      // start recording
      await new Promise((resolve) => {
        capture.onclick = resolve;
      });
      recorder.start();
      capture.replaceWith(stopCapture);
      // use a promise to tell it to stop recording
      await new Promise((resolve) => stopCapture.onclick = resolve);
      recorder.stop();

      let recData = await new Promise((resolve) => recorder.ondataavailable = resolve);
      let arrBuff = await recData.data.arrayBuffer();
      
      // stop the stream and remove the video element
      stream.getVideoTracks()[0].stop();
      div.remove();

      let binaryString = "";
      let bytes = new Uint8Array(arrBuff);
      bytes.forEach((byte) => {
        binaryString += String.fromCharCode(byte);
      })
      return btoa(binaryString);
    }
    """)
  try:
    display(js)
    data = eval_js('recordVideo({})')
    binary = b64decode(data)
    with open(filename, "wb") as video_file:
      video_file.write(binary)
    print(
        f"Finished recording video. Saved binary under filename in current working directory: {filename}"
    )
  except Exception as err:
      # In case any exceptions arise
      print(str(err))
  return filename

In [ ]:
# Run the function, get the video path as saved in your notebook, and play it back here.
from IPython.display import HTML
from base64 import b64encode

video_width = 300

video_path = record_video()
video_file = open(video_path, "r+b").read()

video_url = f"data:video/mp4;base64,{b64encode(video_file).decode()}"
HTML(f"""<video width={video_width} controls><source src="{video_url}"></video>""")

In [ ]:
vidcap = cv2.VideoCapture('video.mp4')
success, image = vidcap.read()
count = 1

if not os.path.exists('videos'):
    os.makedirs('videos')

# Get each individual frame from the video file.
while success:
  cv2.imwrite("./videos/image_%d.jpg" % count, image)    
  success, image = vidcap.read()
  print('Saved image ', count)
  count += 1

In [ ]:
joint_list = []

# Detectron works better the more of you that is in frame,
# so we can trim the video by giving the specific range.

# Process each 
for j in range (80, 180):
  im = cv2.imread("videos/image_"+str(j)+".jpg")

  # Detectron setup.
  cfg = get_cfg()   # get a fresh new config
  cfg.merge_from_file(model_zoo.get_config_file("COCO-Keypoints/keypoint_rcnn_R_50_FPN_3x.yaml"))
  cfg.MODEL.ROI_HEADS.SCORE_THRESH_TEST = 0.7  # set threshold for this model
  cfg.MODEL.WEIGHTS = model_zoo.get_checkpoint_url("COCO-Keypoints/keypoint_rcnn_R_50_FPN_3x.yaml")
  predictor = DefaultPredictor(cfg)
  
  if predictor(im):
    outputs = predictor(im)
    v = Visualizer(im[:,:,::-1], MetadataCatalog.get(cfg.DATASETS.TRAIN[0]), scale=1.2)
    out = v.draw_instance_predictions(outputs["instances"].to("cpu"))

    # Show image with skeleton
    cv2_imshow(out.get_image()[:, :, ::-1])

    if(outputs["instances"]):
      rv = outputs["instances"].pred_keypoints[0]

      xList = []
      yList = []

      for x in rv:
        newX = re.findall(r'\b\d+\b', str(x[0]))
        newY = re.findall(r'\b\d+\b', str(x[1]))
        xList.append(int(newX[0]))
        yList.append(int(newY[0]))

      '''
      X,Y List order => 
      [nose, right eye, left eye, r-ear, l-ear,
      r-shoulder, l-shoulder, r-elbow, l-elbow, r-wrist, l-wrist, 
      r-hip, l-hip, r-knee, l-knee, r-ankle, l-ankle]
      '''

      nose = [xList[0],yList[0]]
      r_eye = [xList[1],yList[1]]
      l_eye = [xList[2],yList[2]]
      r_ear = [xList[3],yList[3]]
      l_ear = [xList[4],yList[4]]

      # Limb motion.
      left_elbow = math.radians(angle_between((xList[6], yList[6]),(xList[8], yList[8]),(xList[10], yList[10])))
      left_shoulder = math.radians(angle_between((xList[5], yList[5]),(xList[6], yList[6]),(xList[8], yList[8])))
      right_elbow = math.pi - math.radians(angle_between((xList[5], yList[5]),(xList[7], yList[7]),(xList[9], yList[9])))
      right_shoulder = math.pi- math.radians(angle_between((xList[6], yList[6]),(xList[5], yList[5]),(xList[7], yList[7])))
      
      # Head up and down, .5 is scaling.
      head_elevation = -.5*(math.pi - math.radians(angle_between(l_ear, nose, r_ear)))
      
      # Head left and right.
      r_ear_eye_len = np.linalg.norm(np.asarray(r_ear) - np.asarray(r_eye))
      l_ear_eye_len = np.linalg.norm(np.asarray(l_ear) - np.asarray(l_eye))
      r_eye_nose = np.linalg.norm(np.asarray(nose) - np.asarray(r_eye))
      l_eye_nose = np.linalg.norm(np.asarray(nose) - np.asarray(l_eye))
      head_azimuth = 1.24*(r_ear_eye_len - l_ear_eye_len )/r_ear_eye_len
      if(abs(head_azimuth) > 1):
        head_azimuth = -1.24*np.sign(head_azimuth)

      joint_list.append([right_shoulder, right_elbow, left_shoulder, left_elbow, head_elevation, head_azimuth])

print(joint_list)

# order = ["right_shoulder", "right_elbow", "left_shoulder", "left_elbow", "head_elevation", "head_azimuth"]

with open('movements.csv', 'w') as f: 
    write = csv.writer(f) 
    # write.writerow(order) 
    write.writerows(joint_list)

print('Written to movements.csv')
